In [ ]:
# graduation project code 2021/2022 Biomedical engineering department , Helwan university  

In [ ]:
# team members : Amr mostafa ,Fady Mostafa ,Sara Ahmed

In [ ]:
# import lib

In [3]:
from sklearn.model_selection import train_test_split
from glob import glob
from scipy.signal import butter, lfilter, convolve, boxcar
from scipy.signal import freqz
from scipy.fftpack import fft, ifft
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler,MinMaxScaler

import numpy as np
import pywt
import matplotlib.pyplot as plt
import pandas as pd
import scipy
import os

# importing required libraries # to form RGB
from PIL import Image
import numpy as np

In [ ]:
#import numpy as np
import scipy as scipy
from numpy.linalg import eig
from scipy.sparse.linalg import eigs
from scipy.linalg import eigh
from numpy.linalg import inv

In [ ]:
# functions

In [ ]:
def butter_bandpass_filter(data, lowcut, highcut, fs):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq

    b, a = butter(12, [low, high], btype='band')
    y = lfilter(b, a, data)
    return y

In [ ]:
# normalization function
scaler= StandardScaler()
def data_normaliz(X):
    X_prep=scaler.fit_transform(X)
    #do here your preprocessing
    return X_prep

In [ ]:
# Function for commone spatial pattern
# Input for function should be in form of array
def CSP(X1,X2):

	S1=np.cov(X1)
	S2=np.cov(X2)
# Solve the eigenvalue problem S1·W = l·S2·W
	L,W=scipy.linalg.eigh(S1, S1+S2) # Mixing matrix W
#	W=W*-1
#	Lambda=np.diag(L)   # Eigenvalues
#	A=np.linalg.inv(W)  # Demixing matrix

#CSP filtered signal is computed as: X_csp = W'*X;
	return L,W

In [ ]:
# Reading Data

In [ ]:
GS1 = pd.read_csv('GS1.csv')

In [ ]:
GS2 = pd.read_csv('GS2.csv')

In [ ]:
GS=GS1.append(GS2)

In [ ]:
from pathlib import Path  
filepath = Path('E:\GP/GS.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True)  
GS.to_csv(filepath)

In [ ]:
GS=GS.drop(['id','Unnamed: 0','FirstDigitTouch' ], axis=1)#remove id

In [ ]:
GS.head()

In [ ]:
GS=np.asarray(GS.astype(float))

In [ ]:
GS

In [4]:
RP1 = pd.read_csv('Replace.csv')

In [5]:
RP2 = pd.read_csv('Replace2.csv')

In [6]:
RP=RP1.append(RP2)

In [7]:
from pathlib import Path  
filepath = Path('E:\GP/RP.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True)  
RP.to_csv(filepath)

In [ ]:
RP=RP1.drop(['id','Unnamed: 0','Replace' ], axis=1) #remove id

In [ ]:
RP.head()

In [ ]:
RP=np.asarray(RP.astype(float))

In [ ]:
RP

In [ ]:
# preprocessing (filter+normalization)

In [ ]:
# GS processing
GS_filter=butter_bandpass_filter(GS,7,30,500)

In [ ]:
GS_filter_normaliz=data_normaliz(GS_filter)

In [ ]:
GS_filter_normaliz=GS_filter_normaliz.T# to be ready for csp [c*t]

In [ ]:
GS_filter_normaliz.shape

In [ ]:
# RP processing
RP_filter=butter_bandpass_filter(RP,7,30,500)

In [ ]:
RP_filter_normaliz=data_normaliz(RP_filter)

In [ ]:
RP_filter_normaliz=RP_filter_normaliz.T # to be ready for csp [c*t]

In [ ]:
RP_filter_normaliz.shape

In [ ]:
# COMMON SPATIAL FILTER CSP(C3,CZ,C4)
# L Eigenvaluses
# w Eigenvector
# GS_filter_normaliz is positive class
# RP_filter_normaliz is negative class
L1,W1=CSP(GS_filter_normaliz[0:3,:],RP_filter_normaliz[0:3,:]) # Mixing matrix W

In [ ]:
# Grasp Signal of electrode (C3.CZ,C4) after CSP
GS_c_csp=np.dot (W1.T,GS_filter_normaliz[0:3,:])

In [ ]:
GS_c_csp.shape

In [ ]:
# Replace Signal of electrode (C3.CZ,C4) after CSP
RP_c_csp=np.dot (W1.T,RP_filter_normaliz[0:3,:])

In [ ]:
RP_c_csp.shape

In [ ]:
# COMMON SPATIAL FILTER CSP(P3,PZ,P4)
# L Eigenvaluses
# w Eigenvector
# GS_filter_normaliz is positive class
# RP_filter_normaliz is negative class
L2,W2=CSP(GS_filter_normaliz[3:6,:],RP_filter_normaliz[3:6,:]) # Mixing matrix W

In [ ]:
# Grasp Signal of electrode (P3.PZ,P4) after CSP
GS_p_csp=np.dot (W2.T,GS_filter_normaliz[3:6,:])

In [ ]:
GS_p_csp.shape

In [ ]:
# Replace Signal of electrode (P3.PZ,P4) after CSP
RP_p_csp=np.dot (W2.T,RP_filter_normaliz[3:6,:])

In [ ]:
RP_p_csp.shape

In [ ]:
############### wavelet transformation ################

In [ ]:
!pip install scaleogram

In [ ]:
import scaleogram as scg
import pywt
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
#cwt=scg.CWT(signal,scales='wavelet')
#Argument 
#signal=sig
#scles= np.arange(1, 70)# as paper mention for dimension of sclogeame
#wavelet='morl'# morlyt as the most common used in EEG signal

In [ ]:
scles= np.arange(1, 70)# as paper mention for dimension of sclogeame
# wavelet='morl'# morlyt as the most common used in EEG signal

In [ ]:
##### creat scalograme for (C3,CZ,C4) GS v.s RP ####

In [ ]:
##### Scalogram for Grasp event (C3,CZ,C4) #####

In [ ]:
# scalogram for event Grasp electrode C3
coef_GS_c3, freqs = pywt.cwt(GS_c_csp[0,:], scles,'morl')

In [ ]:
coef_GS_c3.shape

In [ ]:
# Take 400 sampels
scalogram_GS_c3_1=coef_GS_c3[:,0:400]

In [ ]:
scalogram_GS_c3_1.shape

In [ ]:
# first gray scalogram image C3 ,Grasp 69*400

plt.matshow(scalogram_GS_c3_1,cmap='gray')
plt.colorbar();

In [ ]:
# scalogram for event Grasp electrode Cz
coef_GS_cz, freqs = pywt.cwt(GS_c_csp[1,:], scles,'morl')

In [ ]:
coef_GS_cz.shape

In [ ]:
# Take 400 sampels
scalogram_GS_cz_1=coef_GS_cz[:,0:400]

In [ ]:
scalogram_GS_cz_1.shape

In [ ]:
# first gray scalogram image Cz ,Grasp 69*400

plt.matshow(scalogram_GS_cz_1,cmap='gray')
plt.colorbar();

In [ ]:
# scalogram for event Grasp electrode C4
coef_GS_c4, freqs = pywt.cwt(GS_c_csp[2,:], scles,'morl')

In [ ]:
coef_GS_c4.shape

In [ ]:
# Take 400 sampels
scalogram_GS_c4_1=coef_GS_c4[:,0:400]

In [ ]:
scalogram_GS_c4_1.shape

In [ ]:
# first gray scalogram image C4 ,Grasp 69*400

plt.matshow(scalogram_GS_c4_1,cmap='gray')
plt.colorbar();

In [ ]:
# make resize for sclogram Graspe event  (C3,Cz,C4)

In [ ]:
!pip install scipy

In [ ]:
import scipy.misc

In [ ]:
!pip install Pillow


In [ ]:
from PIL import Image


In [ ]:
pip install scipy==1.1.0

In [ ]:
import imageio

In [ ]:
pip install opencv-python


In [ ]:
from PIL import Image

#Create an Image Object from an Image
im = Image.open(scalogram_GS_c3_1)

#Display actual image
im.show()

#Make the new image half the width and half the height of the original image
resized_im = im.resize(224,224)

#Display the resized imaged
resized_im.show()

#Save the cropped image
resized_im.save('resizedBeach1.jpg')

In [ ]:

#C3#
scalogram_GS_c3_1=Image.resize(scalogram_GS_c3_1,(224,224))#  resize by numpy 
#scalogram_GS_c3_1= imresize(scalogram_GS_c3_1, (224, 224),interp = 'cubic')

In [ ]:
scalogram_GS_c3_1.shape

In [ ]:
# first gray scalogram image C3 ,Grasp after resize (224*224)

plt.matshow(scalogram_GS_c3_1)
plt.colorbar();

In [ ]:
#Cz#
scalogram_GS_cz_1= np.resize(scalogram_GS_cz_1, (224, 224))

In [ ]:
scalogram_GS_cz_1.shape

In [ ]:
# first gray scalogram image Cz ,Grasp after resize (224*224)

plt.matshow(scalogram_GS_cz_1)
plt.colorbar();

In [ ]:
#C4#
scalogram_GS_c4_1= np.resize(scalogram_GS_c4_1, (224, 224))

In [ ]:
scalogram_GS_c4_1.shape

In [ ]:
# first gray scalogram image C4 ,Grasp after resize (224*224)

plt.matshow(scalogram_GS_c4_1)
plt.colorbar();

In [ ]:
# # creating rgb image from these three channel
r=scalogram_GS_c3_1
g=scalogram_GS_cz_1
b=scalogram_GS_c4_1
RGB_GS_C_1 = np.dstack((r,g,b))


In [ ]:
RGB_GS_C_1.shape

In [ ]:
RGB_GS_C_1

In [ ]:
plt.imshow(RGB_GS_C_1)
plt.colorbar();

In [ ]:
# save images as jpg
imsave('RGB_GS_C_1.jpg', RGB_GS_C_1)

In [ ]:
##### Scalogram for Grasp event (P3,PZ,P4) #####

In [ ]:
# scalogram for event Grasp electrode P3
coef_GS_p3, freqs = pywt.cwt(GS_p_csp[0,:], scles,'morl')

In [ ]:
coef_GS_p3.shape

In [ ]:
# Take 400 sampels
scalogram_GS_p3_1=coef_GS_p3[:,0:400]

In [ ]:
scalogram_GS_p3_1.shape

In [ ]:
# first gray scalogram image p3 ,Grasp 69*400

plt.matshow(scalogram_GS_p3_1,cmap='gray')
plt.colorbar();

In [ ]:
# scalogram for event Grasp electrode Pz
coef_GS_pz, freqs = pywt.cwt(GS_p_csp[1,:], scles,'morl')

In [ ]:
coef_GS_pz.shape

In [ ]:
# Take 400 sampels
scalogram_GS_pz_1=coef_GS_pz[:,0:400]

In [ ]:
scalogram_GS_pz_1.shape

In [ ]:
# first gray scalogram image pz ,Grasp 69*400

plt.matshow(scalogram_GS_pz_1,cmap='gray')
plt.colorbar();

In [ ]:
# scalogram for event Grasp electrode P4
coef_GS_p4, freqs = pywt.cwt(GS_p_csp[2,:], scles,'morl')

In [ ]:
coef_GS_p4.shape

In [ ]:
# Take 400 sampels
scalogram_GS_p4_1=coef_GS_p4[:,0:400]

In [ ]:
# first gray scalogram image p4 ,Grasp 69*400

plt.matshow(scalogram_GS_p4_1,cmap='gray')
plt.colorbar();

In [ ]:
#P3#
# scalogram_GS_c3_1=np.resize(scalogram_GS_p3_1,(224,224))#  resize by numpy 
scalogram_GS_p3_1= imresize(scalogram_GS_p3_1, (224, 224),interp = 'cubic')

In [ ]:
scalogram_GS_p3_1.shape

In [ ]:
# first gray scalogram image P3 ,Grasp after resize (224*224)

plt.matshow(scalogram_GS_p3_1)
plt.colorbar();

In [ ]:
#Pz#
# scalogram_GS_c3_1=np.resize(scalogram_GS_p3_1,(224,224))#  resize by numpy 
scalogram_GS_pz_1= imresize(scalogram_GS_pz_1, (224, 224),interp = 'cubic')

In [ ]:
scalogram_GS_pz_1.shape

In [ ]:
# first gray scalogram image Pz ,Grasp after resize (224*224)

plt.matshow(scalogram_GS_pz_1)
plt.colorbar();

In [ ]:
#P4#
# scalogram_GS_c3_1=np.resize(scalogram_GS_p3_1,(224,224))#  resize by numpy 
scalogram_GS_p4_1= imresize(scalogram_GS_p4_1, (224, 224),interp = 'cubic')

In [ ]:
scalogram_GS_p4_1.shape

In [ ]:
# first gray scalogram image P4 ,Grasp after resize (224*224)

plt.matshow(scalogram_GS_p4_1)
plt.colorbar();

In [ ]:
# # creating rgb image from these three channel
r1=scalogram_GS_p3_1
g1=scalogram_GS_pz_1
b1=scalogram_GS_p4_1
RGB_GS_P_1 = np.dstack((r1,g1,b1))

In [ ]:
RGB_GS_P_1.shape

In [ ]:
RGB_GS_P_1

In [ ]:
plt.imshow(RGB_GS_P_1)
plt.colorbar();

In [ ]:
# save images as jpg
imsave('RGB_GS_P_1.jpg', RGB_GS_P_1)